In [1]:
import tensorflow as tf
import numpy as np
import utils

In [2]:
operator = 'add'
(input_train, input_dev, input_test, 
           target_train, target_dev, target_test) = utils.import_data(operator)

In [3]:
# If the training dataset takes all examples, then the dev and test datasets are the same as the training one. 
if input_dev.shape[0] == 0:
    input_dev = input_train
    target_dev = target_train
    input_test = input_train
    target_test = target_train

In [4]:
print(input_train.shape)
print(input_dev.shape)
print(input_test.shape)
print(target_train.shape)
print(target_dev.shape)
print(target_test.shape)

(65536, 16)
(65536, 16)
(65536, 16)
(65536, 9)
(65536, 9)
(65536, 9)


In [5]:
run_id = '20181001180511'
operation = 'add'

parameters = utils.import_parameters(run_id, operation)

w1_value = parameters['h1/kernel']
b1_value = parameters['h1/bias']
b1_value = b1_value.reshape((-1, b1_value.shape[0]))

w2_value = parameters['last_logits/kernel']
b2_value = parameters['last_logits/bias']
b2_value = b2_value.reshape((-1, b2_value.shape[0]))

In [6]:
def test_graph(str_activation_function):
    '''
    str_activation_function: 'sigmoid', 'tlu'
    '''    
    w1 = tf.Variable(w1_value)
    b1 = tf.Variable(b1_value)
    w2 = tf.Variable(w2_value)
    b2 = tf.Variable(b2_value)

    inputs = tf.placeholder(tf.float32, shape=(None, input_train.shape[1]), name='inputs') # None for mini-batch size
    targets = tf.placeholder(tf.float32, shape=(None, target_train.shape[1]), name='targets')

    if str_activation_function == 'tlu':
        h1          = utils.tf_tlu(tf.matmul(inputs, w1) + b1)
        predictions = utils.tf_tlu(tf.matmul(h1, w2) + b2)

    if str_activation_function == 'sigmoid':
        h1          = tf.sigmoid(tf.matmul(inputs, w1) + b1)
        predictions = utils.tf_tlu(tf.sigmoid(tf.matmul(h1, w2) + b2))

    # Accuracy
    (accuracy, n_wrong, n_correct) = utils.get_measures(targets, predictions)
    
    # Run area #########################
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    init = tf.global_variables_initializer()
    
    with tf.Session(config=config) as sess:
        sess.run(init)

        # Run computing test loss, accuracy
        accuracy_value,  n_wrong_value, predictions_value = sess.run(
            [accuracy, n_wrong, predictions],
            feed_dict={inputs:input_test, targets:target_test})
        
    
    return (accuracy_value, n_wrong_value)

In [7]:
(accuracy_value, n_wrong_value) =  test_graph('sigmoid')
print('accuracy: {}, n_wrong_value: {}'.format(accuracy_value, n_wrong_value))

accuracy: 1.0, n_wrong_value: 0


In [8]:
(accuracy_value, n_wrong_value) =  test_graph('tlu')
print('accuracy: {}, n_wrong_value: {}'.format(accuracy_value, n_wrong_value))

accuracy: 0.8299407958984375, n_wrong_value: 11145
